In [2]:
from pyspark.sql import SparkSession

# Create a DataFrame using SparkSession
spark = (SparkSession
    .builder
    .appName("data_cleaning")
    .enableHiveSupport()
    .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/28 15:27:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/28 15:27:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Sorry i seriouly don't know why i have to do this
import pandas as pd

pandas_df = pd.read_json('/home/jazzdung/Projects/E-Commerce-Support-System/mr_clean/shopee.ndjson', lines=True)
df = spark.createDataFrame(pandas_df) 

# # File location and type
# file_location = "/home/jazzdung/Projects/E-Commerce-Support-System/mr_clean/shopee.ndjson"
# file_type = "json"

# # CSV options
# infer_schema = "false"
# first_row_is_header = "true"
# delimiter = ","

# # The applied options are for CSV files. For other file types, these will be ignored.
# df = spark.read.format(file_type) \
#   .option("inferSchema", infer_schema) \
#     .option("mode", "failFast")\
#     .option("multiline", "true")\
#     .load(file_location)
# #   .option("header", first_row_is_header) \
# #   .option("sep", delimiter) \
# #   .load(file_location)

# df.show(100, truncate=False)

/home/jazzdung/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/jazzdung/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [4]:
df.printSchema()

root
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)
 |-- product_desc: string (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_review: string (nullable = true)
 |-- num_sold: string (nullable = true)
 |-- attrs: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- url: string (nullable = true)



**Chú ý: với các data dạng string, cần bỏ các kí tự đặc biệt (chỉ dữ lại chữ + số)**

**Tên sp**
- Dùng regex để lọc và bỏ những cái string có thể leak giá ra ngoài)
- Lowercase + bỏ kí tự đặc biệt

In [5]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
import re

# Lowercase
def lower_case(s):
    return s.lower()

# Remove Yeu thich, Yeu thich+
def remove_liked_product(s):
    s = re.sub('yêu thích\n', '', s)
    s = re.sub('yêu thích\+\n', '', s)
    return s

# Remove contents inside [], option indicate promotion, prices    
def remove_between_square_brackets(s):
    s = re.sub(r'\[.*?\]', '', s)
    return s

# Remove special character
def remove_special_character(s):
    s = re.sub('[^A-Za-z0-9 ]+', ' ', s)
    s = re.sub(' +', ' ', s)
    return s

# Remove vietnamese character
def no_accent_vietnamese(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

# Remove redundant spaces
def remove_spaces(s):
    s = s.lstrip()
    return s

# Bundled up
def clean_product_name(name):
    name = lower_case(name)
    name = remove_liked_product(name)
    name = remove_between_square_brackets(name)
    name = no_accent_vietnamese(name)
    name = remove_special_character(name)
    name = remove_spaces(name)

    return name

udf_clean_product_name = udf(clean_product_name)

df.withColumn("product_name", udf_clean_product_name(df["product_name"])).select("product_name").show(30, truncate=False)


+-------------------------------------------------------------+
|product_name                                                 |
+-------------------------------------------------------------+
|ao khoac cardigan vien xanh nau frmlk form rong chew         |
|ao khoac cardigan vien xanh nau frmlk form rong chew         |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len tam hang quang chau   |
|ao len tam co 3 phan ao len tam ao len 

**Giá sp**
- Bỏ chữ “đ" (đ55.000)

In [6]:
# 50.000đ -> 50.000
def remove_dong(s):
    s = re.sub('₫', '', s)
    return s

# 50.000 -> 50000
def remove_dot(s):
    s = re.sub('\.', '', s)
    return s

# ₫48.000 - ₫59.000 -> ₫48.000
def remove_second_price (s):
    s = s.split('-')[0]
    s = re.sub(' ', '', s)
    return s

def clean_price(s):
    s = remove_dong(s)
    s = remove_dot(s)
    s = remove_second_price(s)
    return s

udf_clean_price = udf(clean_price)

df.withColumn("price", udf_clean_price(df["price"])).select("price").show(10, truncate=False)

+------+
|price |
+------+
|      |
|110000|
|48000 |
|59000 |
|59000 |
|59000 |
|59000 |
|59000 |
|59000 |
|59000 |
+------+
only showing top 10 rows



**Category**    
- Phân category ra làm 3 tầng, sẽ có 3 attribute là cat1,cat2, cat3
- Cat2 có format là :”cat1 / cat2”, cat3 là “cat1 / cat2 / cat3

**Chi tiết sp (dạng String)**
- Phải bỏ html
- Dùng regex để lọc và bỏ những cái string có thể leak giá ra ngoài)
- Lowercase + bỏ kí tự đặc biệt

In [11]:
def remove_between_angle_brackets(s):
    s = re.sub(' &amp;', '', s)
    s = re.sub(r'\<.*?\>', '|', s)
    s = re.sub(r'(.)\1+', r'\1', s)

    # s = re.sub('\n', ' | ', s)
    return s

udf_clean_num_review = udf(remove_between_angle_brackets)


df.withColumn("product_desc_dict", udf_clean_num_review(df["product_desc"])).select("product_desc_dict").show(30, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Số đánh giá**
- 3,2k -> 3200

In [8]:
# 3,2k -> 3200

def k_to_number(s):
    if "," in s:
        s = re.sub(',', '', s)
        s = re.sub('k', '00', s)
    else:
        s = re.sub('k', '000', s)
    
    return s

udf_clean_num_review = udf(k_to_number)

df.withColumn("num_review", udf_clean_num_review(df["num_review"])).select("num_review").show(10, truncate=False)

# df = df.withColumn("num_review", udf_clean_num_review(df["num_review"])).select("num_review")
# df = df.withColumn("num_sold", udf_clean_num_review(df["num_sold"])).select("num_sold")

+----------+
|num_review|
+----------+
|3200      |
|3200      |
|1000      |
|1000      |
|1000      |
|1000      |
|1000      |
|1000      |
|1000      |
|1000      |
+----------+
only showing top 10 rows

